# CS-340 Project Two: Grazioso Salvare Dashboard

In [ ]:
from jupyter_dash import JupyterDash, html, dcc, Input, Output, dash_table
import pandas as pd
import plotly.express as px
import crud_module

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Img(src='assets/Grazioso Salvare Logo.png', style={'width': '200px'}),
    html.H2("SP - Grazioso Salvare Dashboard"),
    html.Div([
        dcc.Input(id='input_user', type='text', placeholder='Enter Username'),
        dcc.Input(id='input_passwd', type='password', placeholder='Enter Password'),
        html.Button('Execute', id='submit-val', n_clicks=0),
    ]),
    html.Hr(),
    html.Div(id='table-out'),
    html.Br(),
    html.Div(id='graph-out')
])

@app.callback(
    [Output('table-out', 'children'),
     Output('graph-out', 'children')],
    [Input('input_user', 'value'),
     Input('input_passwd', 'value'),
     Input('submit-val', 'n_clicks')]
)
def update_output(input_user, input_passwd, n_clicks):
    if n_clicks > 0:
        try:
            db = crud_module.Crud(input_user, input_passwd)
            result = db.read({})
            df = pd.DataFrame(result)
            if df.empty:
                return html.Div("No results found."), ""

            table = dash_table.DataTable(
                data=df.to_dict('records'),
                columns=[{"name": i, "id": i} for i in df.columns],
                page_size=10,
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'left'}
            )

            if 'location_lat' in df.columns and 'location_long' in df.columns:
                fig = px.scatter_mapbox(
                    df,
                    lat="location_lat",
                    lon="location_long",
                    hover_name="name",
                    zoom=9,
                    height=500
                )
                fig.update_layout(mapbox_style="open-street-map")
                graph = dcc.Graph(figure=fig)
            else:
                graph = html.Div("Geolocation data not available.")

            return table, graph

        except Exception as e:
            return html.Div(f"Error: {e}"), ""
    return html.Div("Please enter your credentials and click Execute."), ""

if __name__ == '__main__':
    app.run_server(debug=True)